In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pymorphy2
from collections import Counter

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import tensorflow
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import pandas as pd
import numpy as np
import pickle
import re

In [ ]:
data = pd.read_csv('train_texts.csv')

In [ ]:
print(data.loc[4].text)

I. Отдел литературный. Повести, романы, рассказы, мемуары, стихи и т. д.
II. Критика и библиографические заметки как о русских книгах, так и об иностранных. Сюда же относятся разборы новых пьес, поставленных на наши сцены.
III. Статьи ученого содержания. Вопросы экономические, финансовые, философские, имеющие современный интерес. Изложение самое популярное, доступное и для читателей, не занимающихся специально этими предметами.
IV. Внутренние новости. Распоряжения правительства, события в отечестве, письма из губерний и проч.
V. Политическое обозрение. Полное ежемесячное обозрение политической жизни государств. Известия последней почты, политические слухи, письма иностранных корреспондентов.
VI. Смесь. а) Небольшие рассказы, письма из-за границы и из наших губерний и проч. b) Фельетон. с) Статьи юмористического содержания.
Из этого перечня видно, что все, что может интересовать современного читателя, входит в нашу программу. Из статей юмористического содержания мы сделаем особый отдел 

In [ ]:
data_cleaned = pd.DataFrame(columns=data.columns)
for i in range(data.shape[0]):
  # print(data.loc[i].text)
  cleaned_text =  re.sub('\d+', '', re.sub(r'[^\w\s-]', '', data.loc[i].text.lower()))
  cleaned_text = cleaned_text.replace("\n", " ").replace("- ", " ").replace(" -", " ")
  if cleaned_text[0] == '-':
    cleaned_text = cleaned_text[1:]
  if cleaned_text[-1] == '-':
    cleaned_text = cleaned_text[:-1]
  q = pd.DataFrame({'id': i, 'text': [cleaned_text], 'author': data.loc[i].author})
  # print(q)
  data_cleaned = data_cleaned.append(q)

In [ ]:
data_cleaned = data_cleaned.reset_index().drop(['index'], axis=1)

In [ ]:
print(data_cleaned.loc[0].text)

бабушка вскричала малютка возьми меня с собой я знаю что ты уйдешь как только погаснет спичка уйдешь как теплая печка чудесный жареный гусь и большая славная елка и она поспешно чиркнула всем остатком спичек которые были у нее в руках так ей хотелось удержать бабушку и спички вспыхнули таким ярким пламенем что стало светлее чем днем никогда еще бабушка не была такою красивою такою величественною она взяла девочку на руки и они полетели вместе в сиянии и в блеске высоко-высоко туда где нет ни холода ни голода ни страха к богу в холодный утренний час в углу за домом по-прежнему сидела девочка с розовыми щечками и улыбкой на устах но мертвая она замерзла в последний вечер старого года новогоднее солнце осветило маленький труп девочка сидела со спичками одна пачка почти совсем обгорела она хотела погреться бедняжка говорили люди но никто и не знал что она видела в каком блеске вознеслась вместе с бабушкой к новогодним радостям на небо начнем с того что марли умер в этом нет ни малейшего со

In [ ]:
unique_data = pd.DataFrame(columns=data.columns)
morph = pymorphy2.MorphAnalyzer()
for i in range(data_cleaned.shape[0]):
  print(i)
  cleaned_text = ''
  for k in data_cleaned.loc[i].text.split():
    nf = morph.parse(k)[0].normal_form
    # words = [*filter(lambda x: re.match('[а-я|-][а-я|-]*', x), sorted(set(words)))]
    cleaned_text += nf+' ' if nf not in stopwords.words('russian') else ''
  # print(cleaned_text)
  q = pd.DataFrame({'id': i, 'text': [cleaned_text], 'author': data_cleaned.loc[i].author})
  unique_data = unique_data.append(q)

In [ ]:
unique_data = unique_data.reset_index().drop(['index'], axis=1)

In [ ]:
unique_data

,id,text,author
0,0,бабушка вскричать малютка взять знать уйти пог...,Dostoevsky
1,1,знать скрудж это разумеется знать мочь иначе с...,Dostoevsky
2,2,праздник дядя радость дать бог весь благо земн...,Dostoevsky
3,3,высказать главный передовой мысль наш журнал н...,Dostoevsky
4,4,i отдел литературный повесть роман рассказ мем...,Dostoevsky
...,...,...,...
1729,1729,хотеться канторека классный наставник строгий ...,Remark
1730,1730,это-то заключаться несостоятельность восемнадц...,Remark
1731,1731,дело франц спрашивать кроппа кеммерих опускать...,Remark
1732,1732,мюллер наклоняться принести твой вещь франц ке...,Remark


In [ ]:
authors = list(unique_data['author'].unique())

In [ ]:
all_words = set()
for i in range(unique_data.shape[0]):
  all_words |= set(unique_data.loc[i].text.split())
all_words = sorted(all_words)

In [ ]:
all_words

['-х',
 'a',
 'absolutely',
 'abteilung',
 'abteilungen',
 'accord',
 'action',
 'adieu',
 'admirable',
 'agency',
 'alcohol',
 'alles',
 'aloe',
 'amiga',
 'amstrad',
 'an',
 'and',
 'andrew',
 'angel',
 'animals',
 'antique',
 'ao',
 'apoplecticus',
 'apprehension',
 'assured',
 'atari',
 'aux',
 'avec',
 'avez',
 'b',
 'baboulinka',
 'barbare',
 'baronne',
 'bce',
 'beau',
 'beauty',
 'bebe',
 'bebeть',
 'beloved',
 'best',
 'bete',
 'bettina',
 'bi',
 'blanche',
 'bo',
 'bono',
 'bonus',
 'boy',
 'bête',
 'c',
 'camélias',
 'camеlias',
 'cela',
 'cest',
 'château',
 'cinq',
 'citoyen',
 'closer',
 'cnn',
 'collectioner',
 'comme',
 'common',
 'comprenez',
 'comte',
 'comtesse',
 'connection',
 'coperta',
 'corporation',
 'costa',
 'cousin-маркиз',
 'creative',
 'crescendo',
 'cмотереть',
 'cоветский',
 'cрести',
 'dame',
 'dante',
 'de',
 'decency',
 'decorator',
 'delete',
 'denstchik',
 'deus',
 'dhotoм',
 'dhôtoм',
 'dieu',
 'dolls',
 'donc',
 'dorfbarbier',
 'du',
 'dust',
 'e'

In [ ]:
x_data = {'text': [{}.fromkeys(all_words, 0) for _ in range(unique_data.shape[0])], 'author': [''] * unique_data.shape[0]}
for i in range(unique_data.shape[0]):
  for k in unique_data.loc[i].text.split():
    x_data['text'][i][k] += 1
  x_data['author'][i] = authors.index(unique_data.loc[i].author)

In [ ]:
len(x_data['text'])

1734

In [ ]:
from sklearn import svm

In [ ]:
X = np.array([np.array(list(i.values())) for i in x_data['text']])

In [ ]:
X.shape

(1734, 31165)

In [ ]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
Y = np.array(x_data['author'])

In [ ]:
X_train = X.copy()
y_train = Y.copy()
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.05, random_state=42)

In [ ]:
clf = svm.SVC(decision_function_shape='ovo')
clf.fit(X_train, y_train)

In [ ]:
forest_clf = RandomForestClassifier(random_state=42, n_estimators=830, max_features='auto', max_depth=75, min_samples_split=4, min_samples_leaf=2) # 845 70  900 75
forest_clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=75, min_samples_leaf=2, min_samples_split=4,
                       n_estimators=830, random_state=42)

In [ ]:
result = pd.read_csv('test_texts.csv')

In [ ]:
result

,id,text
0,1734,"Идти ему было немного; он даже знал, сколько ш..."
1,1735,"-Твой дедушка был немножко пиратом, а как увер..."
2,1736,"У меня был немецкий паспорт, годный еще на цел..."
3,1737,"-Ну как знаете,- сказал я и сделал вид, будто ..."
4,1738,Я отмахнулся.\n-На шестнадцать и пятьдесят одн...
...,...,...
325,2059,"-А правда,- сказал Маратик,- давай завтра еще ..."
326,2060,"Иванов (графу). Ax, не зуди ты, зуда! (Львову...."
327,2061,"Существо, которое называли Игорем, захлопнуло ..."
328,2062,"-Ма, вы с дядей Флавием дали мне образование, ..."


In [ ]:
data_test_cleaned = pd.DataFrame(columns=result.columns)
for i in range(result.shape[0]):
  # print(data.loc[i].text)
  cleaned_text =  re.sub('\d+', '', re.sub(r'[^\w\s-]', '', result.loc[i].text.lower()))
  cleaned_text = cleaned_text.replace("\n", " ").replace("- ", " ").replace(" -", " ")
  if cleaned_text[0] == '-':
    cleaned_text = cleaned_text[1:]
  if cleaned_text[-1] == '-':
    cleaned_text = cleaned_text[:-1]
  q = pd.DataFrame({'id': i, 'text': [cleaned_text]})
  # print(q)
  data_test_cleaned = data_test_cleaned.append(q)

In [ ]:
data_test_cleaned = data_test_cleaned.reset_index().drop(['index'], axis=1)

In [ ]:
data_test_cleaned.loc[0].text

'идти ему было немного он даже знал сколько шагов от ворот его дома ровно семьсот тридцать как-то раз он их сосчитал когда уж очень размечтался в то время он и сам еще не верил этим мечтам своим и только раздражал себя их безобразною но соблазнительною дерзостью теперь же месяц спустя он уже начинал смотреть иначе и несмотря на все поддразнивающие монологи о собственном бессилии и нерешимости безобразную мечту как-то даже поневоле привык считать уже предприятием хотя все еще сам себе не верил он даже шел теперь делать пробу своему предприятию и с каждым шагом волнение его возрастало все сильнее и сильнее с замиранием сердца и нервною дрожью подошел он к преогромнейшему дому выходившему одною стеной на канаву а другою в-ю улицу этот дом стоял весь в мелких квартирах и заселен был всякими промышленниками  портными слесарями кухарками разными немцами девицами живущими от себя мелким чиновничеством и проч входящие и выходящие так и шмыгали под обоими воротами и на обоих дворах дома тут слу

In [ ]:
unique_test_data = pd.DataFrame(columns=result.columns)
morph = pymorphy2.MorphAnalyzer()
for i in range(data_test_cleaned.shape[0]):
  print(i)
  cleaned_text = ''
  for k in data_test_cleaned.loc[i].text.split():
    nf = morph.parse(k)[0].normal_form
    cleaned_text += nf+' ' if nf not in stopwords.words('russian') else ''
  # print(cleaned_text)
  q = pd.DataFrame({'id': i, 'text': [cleaned_text]})
  unique_test_data = unique_test_data.append(q)

In [ ]:
unique_test_data = unique_test_data.reset_index().drop(['index'], axis=1)

In [ ]:
x_data = {'text': [{}.fromkeys(all_words, 0) for _ in range(unique_test_data.shape[0])]}
for i in range(unique_test_data.shape[0]):
  for k in unique_test_data.loc[i].text.split():
    if k in x_data['text'][i]:
      x_data['text'][i][k] += 1
  # x_data['author'][i] = authors.index(unique_test_data.loc[i].author)

In [ ]:
X_ = np.array([np.array(list(i.values())) for i in x_data['text']])

In [ ]:
X_.shape

(330, 31165)

In [ ]:
X_

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
prediction = forest_clf.predict(X_)

In [ ]:
res_pred = pd.DataFrame(columns=['id', 'author'])
for i in range(result.shape[0]):
  res_pred = res_pred.append(pd.DataFrame({'id' : result.loc[i].id, 'author': [authors[prediction[i]]]}))

In [ ]:
res_pred

,id,author
0,1734,Dostoevsky
0,1735,Akunin
0,1736,Akunin
0,1737,Akunin
0,1738,Akunin
...,...,...
0,2059,Bulychev
0,2060,Akunin
0,2061,Akunin
0,2062,Pratchett


In [ ]:
res_pred = res_pred.set_index('id')

In [ ]:
with open('out.csv', 'w') as f:
  f.write(res_pred.to_csv())

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
lr = LogisticRegression(C=40.0, random_state=1, solver='lbfgs', multi_class='ovr')

In [ ]:
lr.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


LogisticRegression(C=40.0, multi_class='ovr', random_state=1)

In [ ]:
# y_predict = lr.predict(X_test)
  
# # Use metrics.accuracy_score to measure the score
# print("LogisticRegression Accuracy %.3f" % accuracy_score(y_test, y_predict))

In [ ]:
X_

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
pred = lr.predict(X_)

In [ ]:
res_pred = pd.DataFrame(columns=['id', 'author'])
for i in range(result.shape[0]):
  res_pred = res_pred.append(pd.DataFrame({'id' : result.loc[i].id, 'author': [authors[pred[i]]]}))

In [ ]:
res_pred

,id,author
0,1734,Dostoevsky
0,1735,Pratchett
0,1736,Akunin
0,1737,King
0,1738,Akunin
...,...,...
0,2059,Bulychev
0,2060,Chehov
0,2061,Pratchett
0,2062,Pratchett


In [ ]:
res_pred = res_pred.set_index('id')

In [ ]:
res_pred

,author
id,
1734,Dostoevsky
1735,Pratchett
1736,Akunin
1737,King
1738,Akunin
...,...
2059,Bulychev
2060,Chehov
2061,Pratchett


In [ ]:
with open('out.csv', 'w') as f:
  f.write(res_pred.to_csv())